In [1]:
import gspread
from google.oauth2.service_account import Credentials


In [3]:
gc = gspread.service_account(filename="D:/agung/OneDrive - db/pribari/bimbingin_dong/assignment/assignment_day_12/airflow/dags/google_secret.json")
sh = gc.open_by_key("1CDqjVawY8aKLQD6ycQoQ6mUQ4TSbdrO5lh6MuxdaLhQ").sheet1()
sh

TypeError: 'Worksheet' object is not callable

In [15]:
sh.append_row([''])

{'spreadsheetId': '1CDqjVawY8aKLQD6ycQoQ6mUQ4TSbdrO5lh6MuxdaLhQ',
 'tableRange': 'Production!A1:N1',
 'updates': {'spreadsheetId': '1CDqjVawY8aKLQD6ycQoQ6mUQ4TSbdrO5lh6MuxdaLhQ',
  'updatedRange': 'Production!A2',
  'updatedRows': 1,
  'updatedColumns': 1,
  'updatedCells': 1}}

In [ ]:
def etl_to_gspread():

    # Path to the JSON credentials file
    json_credentials_file = 'google_secret.json'

    # Google Sheets authentication parameters
    spreadsheet_id = '1CDqjVawY8aKLQD6ycQoQ6mUQ4TSbdrO5lh6MuxdaLhQ'

    # Authenticate with Google Sheets and open the spreadsheet
    gc = gspread.service_account(filename=json_credentials_file)
    sh = gc.open_by_key(spreadsheet_id).sheet1()

    #add value in specific sheet 
    sh.append_row([''])

dag = DAG(
    dag_id = 'chinook_etl',
    schedule_interval='0 2 * * *',  # At 02:00 WIB every day
    start_date=datetime(2024, 2, 23),
    catchup=False
)

# Task to load database
load_to_postgres = PostgresOperator(
    task_id='load_to_postgres',
    postgres_conn_id='chinook',
    sql="""
    SELECT 
    *,
    'Wisnu' as Student Name 
    FROM tracks
    """,
    dag=dag
)

# Task to extract data from PostgreSQL and load to Google Sheets
etl_to_gspread_task = PythonOperator(
    task_id='etl_to_gspread',
    python_callable=etl_to_gspread,
    dag=dag
)

# Set dependencies
load_to_postgres >> etl_to_gspread_task